In [ ]:
import datajoint as dj
from ibl_pipeline import reference, subject, action, acquisition, data, behavior
import matplotlib.pyplot as plt

## Check weights of a subject

In [ ]:
subj = subject.Subject & 'subject_nickname="IBL-T1"'
subj

In [ ]:
weighing_time, weight = (action.Weighing & subj).fetch('weighing_time', 'weight')

In [ ]:
weighing_date = [time.date() for time in weighing_time]

In [ ]:
plt.plot(weighing_date, weight, 'o-')
plt.xticks(rotation='vertical')

In [ ]:
weighing_with_date = action.Weighing.proj(weighing_date='DATE(weighing_time)')
weighing_with_date

In [ ]:
weighing_date, weight = (weighing_with_date * action.Weighing & subj).fetch('weighing_date', 'weight')

In [ ]:
plt.plot(weighing_date, weight, 'o-')
plt.xticks(rotation='vertical')

In [ ]:
last_time = subj.aggr(action.Weighing, lastest='MAX(weighing_time)').fetch1('lastest')

In [ ]:
weight = (action.Weighing & {'weighing_time': last_time}).fetch1('weight')
weight

# Query of water type that weekend since September

Step 1: Project the table action.WaterAdministration to only contain the information of interest together with the primary key. 
<br> With `.proj()`, users are able to restrict to certain attributes, rename the existing attributes, or create new attributes computed from the original ones. In this case, we would like to compute the day of week of water administration `water_admin_dow`

In [ ]:
water_admin = action.WaterAdministration.proj(
    'watertype_name', water_admin_dow='dayofweek(administration_time)')
water_admin

Step 2: Restrict to the entries with `administration_time` in the weekend:

In [ ]:
water_admin_weekend = water_admin & 'administration_time>"2018-08-31"' & \
    'water_admin_dow in (6,7)' & subj
water_admin_weekend

Step 3: If you want to put the subject_nickname aside, you could use `*` operator, that allows joining of two tables.

In [ ]:
subj.proj('subject_nickname') * water_admin_weekend

Finally, if you want, you can also do the whole query in a single line:

In [ ]:
(subject.Subject & 'subject_nickname="IBL-T1"').proj('subject_nickname') * (action.WaterAdministration.proj('watertype_name', dow='dayofweek(administration_time)') & 'administration_time>"2018-08-31"' & 'dow in (6,7)')

# Query of water type on Monday, Tuesday and Friday

Similarly as above:

In [ ]:
subj.proj('subject_nickname') * \
    (action.WaterAdministration.proj('watertype_name', water_admin_dow='dayofweek(administration_time)') &  \
     'administration_time>"2018-08-31"' & 'water_admin_dow in (1, 2, 5)')

# Query of the weight, water intake and trial counts on Monday, Tuesday and Friday.

### Weight

Project the weight table to contain only the weight and the primary key, add attribute `wt_dow` 

In [ ]:
weight = action.Weighing.proj('weight', wt_dow='dayofweek(weighing_time)')
weight

Restrict to the weight that abtained on Monday, Tuesday and Friday:

In [ ]:
weight_interest = weight & subj & 'weighing_time>"2018-08-31"' & 'wt_dow in (1, 2, 5)'
weight_interest

Similarly, if you would like the `subject_nickname` showing aside, do the join trick again:

In [ ]:
subj.proj('subject_nickname') * weight_interest

Again, the whole query would be done in one line as well, giving the same results:

In [ ]:
(subject.Subject & 'subject_nickname="IBL-T1"').proj('subject_nickname') * (action.Weighing.proj('weight', wt_dow='dayofweek(weighing_time)') & 'weighing_time>"2018-08-31"' & 'wt_dow in (1, 2, 5)')

### Water Intake

In [ ]:
(subject.Subject & 'subject_nickname="IBL-T1"').proj('subject_nickname') * (action.WaterAdministration.proj('watertype_name', water_intake="water_administered", dow='dayofweek(administration_time)') & 'administration_time>"2018-08-31"' & 'dow in (1, 2, 5)')